# Cài đặt môi trường

In [1]:
INSTALL_LIB = False
TIME_SLEEP_FOR_WAITING = 1
SCROLL_PAUSE_TIME = 0.2
PATH_TO_PLAYLIST_CSV = './playlist.csv'
PATH_TO_USER_CSV = './user.csv'
PATH_TO_TRACK_CSV = './track.csv'
MAX_SCROLLING = 500


In [2]:
#Chạy các câu command lines cần thiết cho việc cài đặt các thư viện
if INSTALL_LIB: 
    !pip install requests_html 
    !pip install selenium
    !pip install webdriver_manager
    !pip install pandas
    !pip install seaborn
    !pip install tqdm

In [3]:
#Thêm các thư viện và modules cần thiết
from requests_html import HTMLSession
import requests 
import json
import time
import numpy as np 
import pandas as pd
import datetime as dt
import re
import selenium
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [4]:
#Định nghĩa driver của webdriver
browser = webdriver.Chrome(executable_path='./chromedriver_ubuntu') 
browser.maximize_window()

# Thu thập dữ liệu

In [5]:
#Khai báo đường dẫn đến trang web gốc
root_url = "https://soundcloud.com/discover"
browser.get(root_url)
# Tắt cửa sổ quảng cáo nếu có bằng cách ấn phím Esc 
browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
search_textbox = browser.find_element_by_css_selector('.headerSearch__input')
search_textbox.send_keys('edm')
search_textbox.submit()
browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)

Có thể thấy dữ liệu về chủ đề nhạc EDM rất đa dạng, nhưng em nhận thấy nhưng comments của User trong các tracks chủ yếu là spam comments. Vì vậy em chỉ lấy những Users nào có upload những Track hoặc Playlist lên Soundcloud

In [6]:
#Thu thập thông tin của các playlist
def check_invalid(func):
    def function_wrapper(*args):
        result = ''
        try:
            result = func(*args)
        except:
            pass
        return result
    return function_wrapper

def check_invalid_for_getting_attr(func):
    def function_wrapper(x,y):
        result = ''
        try:
            result = func(x,y)
        except:
            pass
        return result
    return function_wrapper

class BaseClass:
    @check_invalid
    def get_text_from_css_selector(self,selector):
        element = browser.find_element_by_css_selector(selector)
        return element.text if element else ''
    
    @check_invalid
    def get_attribute_from_css_selector(self, selector, attr):
        element = browser.find_element_by_css_selector(selector)
        return element.get_attribute(attr) if element else ''
    def add_new_value(self, url):
        pass
    def get_dict(self):
        pass
class Playlist(BaseClass):
    def __init__(self):
        self.title = []
        self.name_of_user_owner = []
        self.link_of_user_owner = []
        self.uploaded_time = []
        self.number_of_tracks = []
        self.duration_of_all_tracks = []
        self.number_of_likes = []

    def add_new_value(self, url):
        browser.get(url)
        #We need to sleep until page has loaded
        time.sleep(TIME_SLEEP_FOR_WAITING)
        title = self.get_text_from_css_selector('span.soundTitle__title span')
        #title = self.get_attribute_from_css_selector('span.soundTitle__title span', 'text')
        user_owner = self.get_text_from_css_selector('a.soundTitle__username')
        user_owner_link = self.get_attribute_from_css_selector('a.soundTitle__username','href')
        uploaded_time\
        = self.get_attribute_from_css_selector('.fullHero__uploadTime time','title')

        number_of_tracks\
        = self.get_attribute_from_css_selector('.genericTrackCount__title','innerHTML')

        duration_of_all_tracks\
        = self.get_attribute_from_css_selector('div.genericTrackCount__duration','innerHTML')

        number_of_likes\
        = self.get_attribute_from_css_selector('ul.soundStats > li:first-child','title')
        if number_of_likes:
            number_of_likes = number_of_likes.rsplit(' ')[0]
        
        self.title.append(title)
        self.name_of_user_owner.append(user_owner)
        self.link_of_user_owner.append(user_owner_link)
        self.uploaded_time.append(uploaded_time)
        self.number_of_tracks.append(number_of_tracks)
        self.duration_of_all_tracks.append(duration_of_all_tracks)
        self.number_of_likes.append(number_of_likes)
    
    def get_track_list(self, url):
        browser.get(url)
        time.sleep(TIME_SLEEP_FOR_WAITING)
        number_of_tracks\
        = self.get_attribute_from_css_selector('.genericTrackCount__title','innerHTML')
        number_of_tracks = int(number_of_tracks)
        html_element = browser.find_element_by_tag_name('html')
        for i in range(int(number_of_tracks/5)):
            html_element.send_keys(Keys.PAGE_DOWN)
            time.sleep(SCROLL_PAUSE_TIME)

        elements = browser.find_elements_by_css_selector(\
        '.trackItem__content a.trackItem__trackTitle')
        
        track_list_links = []
        for element in elements:
            track_list_links.append(element.get_attribute('href'))
        return track_list_links
    def get_user_owner(self, url):
        browser.get(url)
        time.sleep(TIME_SLEEP_FOR_WAITING)
        return self.get_attribute_from_css_selector(\
        'a.soundTitle__username','href')

    def get_dict(self):
        return {'title': self.title, 'name_of_user_owner': self.name_of_user_owner,\
        'link_of_user_owner': self.link_of_user_owner, 'uploaded_time': self.uploaded_time,\
        'number_of_tracks': self.number_of_tracks, 'duration_of_all_tracks':\
        self.duration_of_all_tracks, 'number_of_likes':self.number_of_likes}

In [7]:
#Thu thập thông tin của các playlist
#for playlist_link in tqdm(playlist_links):
class User(BaseClass):
    def __init__(self):
        self.number_of_reposts = []
        self.number_of_playlists = []
        self.number_of_albums = []
        
        self.name = []
        self.support_name = []
        self.location = []
        self.number_of_followers = []
        self.number_of_following = []
        self.number_of_tracks = []
        self.number_of_likes = []
        self.number_of_commets = []
        self.link_of_user_owner = []
    
    def add_new_value(self, url):
        browser.get(url)
        #We need to sleep until page has loaded
        time.sleep(TIME_SLEEP_FOR_WAITING)
        name = self.get_text_from_css_selector('h3.profileHeaderInfo__userName')
        
        elements = browser.find_elements_by_css_selector('h4.profileHeaderInfo__additional')
        
        support_name = elements[0].text if len(elements) > 0 else ''
        location = elements[1].text if len(elements) > 1 else ''

        number_of_followers = self.get_text_from_css_selector(\
        'td.infoStats__stat:nth-child(1) > a > div.infoStats__value')
        number_of_following = self.get_text_from_css_selector(\
        'td.infoStats__stat:nth-child(2) > a > div.infoStats__value')
        number_of_tracks = self.get_text_from_css_selector(\
        'td.infoStats__stat:nth-child(3) > a > div.infoStats__value')
        list_elements_needed_filter = browser.find_elements_by_css_selector(\
        '.sidebarHeader__actualTitle')
        
        number_of_likes = ''
        number_of_commets = ''
        
        for element in list_elements_needed_filter:
            tmp = element.text
            if not tmp:
                continue
            tmp.strip()
            tail_str = tmp.split(' ')[1]
            if tail_str == 'likes':
                number_of_likes = tmp.split(' ')[0]
            elif tail_str == 'comments':
                number_of_commets = tmp.split(' ')[0]
        self.name.append(name)
        self.support_name.append(support_name)
        self.location.append(location) 
        self.number_of_followers.append(number_of_followers)
        self.number_of_following.append(number_of_following)
        self.number_of_tracks.append(number_of_tracks)
        self.number_of_likes.append(number_of_likes)
        self.number_of_commets.append(number_of_commets)
        self.link_of_user_owner.append(url)
    def get_dict(self):
        return {'name': self.name,\
        'support_name': self.support_name,\
        'location' : self.location,\
        'number_of_follower': self.number_of_followers,\
        'numbert_of_following': self.number_of_following,\
        'number_of_tracks': self.number_of_tracks,\
        'number_of_likes': self.number_of_likes,\
        'number_of_commets': self.number_of_commets,\
        'link_of_user_owner': self.link_of_user_owner}

In [8]:
class Track(BaseClass):
    def __init__(self):
        self.link_singer_belong = []
        self.name_of_track = []
        self.link_playlist_belong = []
        self.uploaded_time = []
        self.number_of_plays = []
        self.number_of_likes = []
        self.number_of_repost = []
        self.number_of_comment = []
    
    def add_new_value(self, url):
        browser.get(url)
        time.sleep(TIME_SLEEP_FOR_WAITING)
        link_singer_belong = self.get_attribute_from_css_selector(\
        '.soundTitle__usernameHeroContainer a', 'href')
        name_of_track = self.get_text_from_css_selector(\
        '.soundTitle__usernameTitleContainer .soundTitle__title span')
        link_playlist_belong = self.get_attribute_from_css_selector(\
        'div.fullHero__parentLink a', 'href') 
        uploaded_time = self.get_attribute_from_css_selector(\
        '.fullHero__uploadTime time', 'title')
        
        list_elements = browser.find_elements_by_css_selector(\
        '.listenEngagement__stats li')
        number_of_plays = 0
        number_of_likes = 0
        number_of_repost = 0
        if len(list_elements) > 2:
            number_of_plays = list_elements[0].get_attribute('title').split(' ')[0]
            number_of_likes = list_elements[1].get_attribute('title').split(' ')[0]
            number_of_repost = list_elements[2].get_attribute('title').split(' ')[0]
        number_of_comment = self.get_text_from_css_selector('.commentsList__actualTitle')\
        .split(' ')[0]
        
        self.link_singer_belong.append(link_singer_belong)
        self.name_of_track.append(name_of_track)
        self.link_playlist_belong.append(link_playlist_belong)
        self.uploaded_time.append(uploaded_time)
        self.number_of_plays.append(number_of_plays)
        self.number_of_likes.append(number_of_likes)
        self.number_of_repost.append(number_of_repost)
        self.number_of_comment.append(number_of_comment)
    def get_user_owner(self, url):
        browser.get(url)
        time.sleep(TIME_SLEEP_FOR_WAITING)
        return self.get_attribute_from_css_selector(\
        '.soundTitle__usernameHeroContainer a', 'href')
    def get_dict(self):
        return {'link_singer_belong': self.link_singer_belong,\
        'name_of_track': self.name_of_track,\
        'link_playlist_belong': self.link_playlist_belong,\
        'uploaded_time': self.uploaded_time,\
        'number_of_plays': self.number_of_plays,\
        'number_of_plays': self.number_of_likes,\
        'number_of_repost': self.number_of_repost,\
        'number_of_comment': self.number_of_comment }


In [9]:
#Test declaration
EXAMPLE_PLAYLIST_LINK = 'https://soundcloud.com/courtney-mccann/sets/edm'
EXAMPLE_USER_LINK = 'https://soundcloud.com/vintageculturemusic'
EXAMPLE_TRACK_LINK = 'https://soundcloud.com/dackjaniels/trees?in=courtney-mccann/sets/edm'

playlist = Playlist()
playlist.add_new_value(EXAMPLE_PLAYLIST_LINK)
print('PLAYLIST TEST')
print(playlist.get_dict())

instance_user = User()
instance_user.add_new_value(EXAMPLE_USER_LINK)
print('USER TEST')
print(instance_user.get_dict())

instance_user = Track()
instance_user.add_new_value(EXAMPLE_TRACK_LINK)
print('TRACK TEST')
print(instance_user.get_dict())

PLAYLIST TEST
{'title': ['Edm'], 'name_of_user_owner': ['Courtney Mccann'], 'link_of_user_owner': ['https://soundcloud.com/courtney-mccann'], 'uploaded_time': ['24 June 2016'], 'number_of_tracks': ['469'], 'duration_of_all_tracks': ['1:50:02'], 'number_of_likes': ['2,014']}
USER TEST
{'name': ['Vintage Culture Pro Unlimited'], 'support_name': ['LukasRuiz'], 'location': [''], 'number_of_follower': ['576K'], 'numbert_of_following': ['367'], 'number_of_tracks': ['233'], 'number_of_likes': ['368'], 'number_of_commets': ['3,295'], 'link_of_user_owner': ['https://soundcloud.com/vintageculturemusic']}
TRACK TEST
{'link_singer_belong': [''], 'name_of_track': [''], 'link_playlist_belong': [''], 'uploaded_time': [''], 'number_of_plays': [0], 'number_of_repost': [0], 'number_of_comment': ['']}


In [10]:
class Crawler():
    def __init__(self):
        self.dict_of_playlists = Playlist()
        self.dict_of_tracks = Track()
        self.dict_of_user = User()
        self.list_of_playlist_link = set()
        self.list_of_track_link = set()
        self.list_of_user_link = set()
    def push_list_of_playlist_from_links(self, url):
        #Thu thập danh sách các playlist liên quan đến edm
        print(f'Getting playlist links from {url}...')
        #self.get_list_of_playlist_links(url)
        try:
            browser.get(url)
            html_element = browser.find_element_by_tag_name('html')
            for i in tqdm(range(MAX_SCROLLING)):
                html_element.send_keys(Keys.PAGE_DOWN)
                time.sleep(SCROLL_PAUSE_TIME)

            elements = browser.find_elements_by_css_selector('a.sound__coverArt')
            for element in elements:
                tmp = element.get_attribute('href')
                if tmp not in self.list_of_playlist_link:
                    self.list_of_playlist_link.add(tmp)
            print('Getting playlist links is successful.')
            print(f'Number of playlist links has got: {len(self.list_of_playlist_link)}')
        except:
            print('Getting playlist links is failed.')
    def run(self):
        print(f'Parsing {str(len(self.list_of_playlist_link))} playlists.......')
        for playlist_link in tqdm(self.list_of_playlist_link):
            try:
                self.dict_of_playlists.add_new_value(playlist_link)
                user_owner_link = self.dict_of_playlists.get_user_owner(playlist_link)
                self.dict_of_user.add_new_value(user_owner_link)
                print(playlist_link)
                list_of_all_track = self.dict_of_playlists.get_track_list(playlist_link)
                print(f'Processing {str(len(list_of_all_track))} track links...')
                for track_link in list_of_all_track:
                    try:
                        if track_link not in self.list_of_track_link:
                            self.list_of_track_link.add(track_link)
                            self.dict_of_tracks.add_new_value(track_link)
                            owner = self.dict_of_tracks.get_user_owner(track_link)
                            if owner not in self.list_of_user_link:
                                self.list_of_user_link.add(owner)
                                self.dict_of_user.add_new_value(owner)
                    except:
                        print('Failed to parse: '+ track_link)
                        pass
            except:
                print('Failed to parse: '+ playlist_link)
                pass
        print(f'Done running.')
            
    def print_all(self):
        print(self.dict_of_playlists.get_dict())
        print(self.dict_of_tracks.get_dict())
        print(self.dict_of_user.get_dict())
    def export_all_to_csv(self):
        playlist_df = pd.DataFrame(self.dict_of_playlists.get_dict())
        playlist_df.to_csv(PATH_TO_PLAYLIST_CSV)
        
        user_df = pd.DataFrame(self.dict_of_user.get_dict())
        user_df.to_csv(PATH_TO_USER_CSV)
        
        track_df = pd.DataFrame(self.dict_of_tracks.get_dict())
        track_df.to_csv(PATH_TO_TRACK_CSV)        


In [ ]:
crawler = Crawler()
crawler.push_list_of_playlist_from_links('https://soundcloud.com/search/sets?q=edm')
crawler.push_list_of_playlist_from_links('https://soundcloud.com/search/sets?q=Electronic%20dance%20music')

Getting playlist links from https://soundcloud.com/search/sets?q=edm...


 51%|█████     | 254/500 [01:17<01:22,  2.98it/s]

In [ ]:
crawler.run()

In [ ]:
crawler.export_all_to_csv()